# Music Recommendation on Spotify Data

This example demonstrates how you can use KDB.AI to perform similarity recommendations using vector embeddings created from both categorical and numeric music data.

Applications like Spotify and YouTube Music perform hundreds of millions of song recommendations for users every single day. They do this by extracting a vast array of features about every given song and artist and comparing their characteristics.
By leveraging this sort of data, KDB.AI can be used to productionize a music recommendation system and help to quickly and efficiently find music similar to given input songs.

### Aim

In this tutorial, we'll break down how you might perform similarity search on music, taking some Spotify data as an example and using KDB.AI as the vector database to store and query this data.
This breaks down as follows:

1. Load Song Data
1. Create Song Vector Embeddings
1. Store Embeddings In KDB.AI
1. Search For Similar Songs To A Target Song
1. Delete the KDB.AI Table

---

## 0. Setup

### Import Packages

We will start by importing all of the Python packages needed to run this music recommendation system example.
This includes packages for reading in the data, embedding it as vectors, and interacting with the vector database.

In [1]:
import pandas as pd
import numpy as np

In [2]:
# embedding categorical data
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec

In [3]:
# vector DB
import os
import kdbai_client as kdbai
from getpass import getpass
import time

### Configure Console

In order to fully view our embeddings when it comes to displaying the results, we must increase the maximum allowed column width in Pandas DataFrames from the default value.

In [4]:
pd.set_option("max_colwidth", 1000)

This removes a warning that appears when performing in-place column assignment.

In [5]:
pd.options.mode.chained_assignment = None

### Define Helper Functions

Defining these two helper functions will allow us to easily show the shape and head of any Pandas DataFrames or embedding arrays passed.

In [6]:
def show_df(df: pd.DataFrame) -> pd.DataFrame:
    print(df.shape)
    return df.head()

In [7]:
def show_embeddings(embeddings: np.array) -> list[int]:
    print("Num Embeddings:", len(embeddings))
    print("Embedding Size:", len(embeddings[0]))
    return list(embeddings[0])

## 1. Load Song Data

The song data we will read in will be taken from an [open-source Spotify dataset](https://www.kaggle.com/datasets/vatsalmavani/spotify-dataset) on Kaggle. There are 5 files on Kaggle, however, only one file is relevant to this analysis.
This dataset contains a list of metadata on 170,000 songs from 1921 to 2020. This metadata includes:
- Song Name
- Artist Name
- Song Year
- Various features about the song's music, including:
    * acousticness
    * danceability
    * duration_ms
    * energy
    * explicit
    * instrumentalness
    * key
    * liveness
    * loudness
    * mode
    * popularity
    * release_date
    * speechiness
    * tempo
    * valence

### Read In The Spotify Data From The CSV

We can read this song data from a CSV into a Pandas DataFrame and show the resulting table.

In [8]:
raw_song_df = pd.read_csv("data/song_data.csv")

In [9]:
show_df(raw_song_df)

(170653, 19)


,id,name,artists,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,release_date,speechiness,tempo,valence,year
0,4BJqT0PrAfrxzMOxytFOIz,"Piano Concerto No. 3 in D Minor, Op. 30: III. Finale. Alla breve","['Sergei Rachmaninoff', 'James Levine', 'Berliner Philharmoniker']",0.982,0.279,831667,0.211,0,0.878000,10,0.665,-20.096,1,4,1921,0.0366,80.954,0.0594,1921
1,7xPhfUan2yNtyFG0cUWkt8,Clancy Lowered the Boom,['Dennis Day'],0.732,0.819,180533,0.341,0,0.000000,7,0.160,-12.441,1,5,1921,0.4150,60.936,0.9630,1921
2,1o6I8BglA6ylDMrIELygv1,Gati Bali,['KHP Kridhamardawa Karaton Ngayogyakarta Hadiningrat'],0.961,0.328,500062,0.166,0,0.913000,3,0.101,-14.850,1,5,1921,0.0339,110.339,0.0394,1921
3,3ftBPsC5vPBKxYSee08FDH,Danny Boy,['Frank Parker'],0.967,0.275,210000,0.309,0,0.000028,5,0.381,-9.316,1,3,1921,0.0354,100.109,0.1650,1921
4,4d6HGyGT8e121BsdKmw9v6,When Irish Eyes Are Smiling,['Phil Regan'],0.957,0.418,166693,0.193,0,0.000002,3,0.229,-10.096,1,2,1921,0.0380,101.665,0.2530,1921


### Pre-process The Data

Here we will perform a few operations on this Pandas DataFrame to get it into the correct format for creating the vector embeddings for our vector database.
This will include:
- Adding a column prefix
- Removing excess columns
- Fixing column values
- Combining columns into one
- Removing duplicate rows

Once these pre-processing steps have been carried out, our data will be clean and in the correct format to start creating embeddings.

In [10]:
# add "song_" prefix to col names
song_df = raw_song_df.add_prefix("song_")

In [11]:
# drop unused cols
song_df = song_df.drop(columns=["song_id", "song_release_date"])

In [12]:
# fix artists list names - remove quotes
def fix_artists(str_list):
    return ", ".join([v for v in str_list.rstrip("']").lstrip("['").split("', '")])


song_df["song_artists"] = song_df["song_artists"].apply(fix_artists)

In [13]:
# combine song_name & song_artists into song_description
song_df.insert(
    0, "song_description", song_df["song_name"] + " - " + song_df["song_artists"]
)

In [14]:
# remove duplicate rows
song_data = song_df[
    ~song_df.duplicated(subset=["song_description"], keep="first")
].reset_index(drop=True)

In [15]:
show_df(song_df)

(170653, 18)


,song_description,song_name,song_artists,song_acousticness,song_danceability,song_duration_ms,song_energy,song_explicit,song_instrumentalness,song_key,song_liveness,song_loudness,song_mode,song_popularity,song_speechiness,song_tempo,song_valence,song_year
0,"Piano Concerto No. 3 in D Minor, Op. 30: III. Finale. Alla breve - Sergei Rachmaninoff, James Levine, Berliner Philharmoniker","Piano Concerto No. 3 in D Minor, Op. 30: III. Finale. Alla breve","Sergei Rachmaninoff, James Levine, Berliner Philharmoniker",0.982,0.279,831667,0.211,0,0.878000,10,0.665,-20.096,1,4,0.0366,80.954,0.0594,1921
1,Clancy Lowered the Boom - Dennis Day,Clancy Lowered the Boom,Dennis Day,0.732,0.819,180533,0.341,0,0.000000,7,0.160,-12.441,1,5,0.4150,60.936,0.9630,1921
2,Gati Bali - KHP Kridhamardawa Karaton Ngayogyakarta Hadiningrat,Gati Bali,KHP Kridhamardawa Karaton Ngayogyakarta Hadiningrat,0.961,0.328,500062,0.166,0,0.913000,3,0.101,-14.850,1,5,0.0339,110.339,0.0394,1921
3,Danny Boy - Frank Parker,Danny Boy,Frank Parker,0.967,0.275,210000,0.309,0,0.000028,5,0.381,-9.316,1,3,0.0354,100.109,0.1650,1921
4,When Irish Eyes Are Smiling - Phil Regan,When Irish Eyes Are Smiling,Phil Regan,0.957,0.418,166693,0.193,0,0.000002,3,0.229,-10.096,1,2,0.0380,101.665,0.2530,1921


## 2. Create Song Vector Embeddings

We will create vector embeddings from this data in three steps:

- A. Encoding the categorical `song_description` column as numeric values
- B. Scaling the numeric column values
- C. Joining these two sets of encodings together into one vector embedding

### A. Embed Categorical Song Metadata

To embed the `song_description` column as numeric vectors, we must perform natural language processing on them.
This involves tokenising the descriptions to break them up into their individual sub-parts and then using a `Word2Vec` model to turn these tokenised song descriptions into vectors.

The length of the vectors we turn these desciptions into is configurable, however, in this case we chose to set this to `15` as there are also 15 numeric columns which describe the song. We do not want to bias the final embedding vectors in favour of either the categorical columns or the numeric columns, so it made sense to keep the number of values representing each the same for both.

<div class="alert alert-block alert-warning">
    <b>Note: </b>
    Before running the following line of code, please ensure that you have installed the English sentence tokenizer as stated in the `README.md` file in this repository.
</div>

In [16]:
# tokenize the descriptions
tokenised_song_descs = [word_tokenize(v.lower()) for v in song_data["song_description"]]

In [17]:
# create embedding model
embedding_dim = 15

word2Vec_model = Word2Vec(
    sentences=tokenised_song_descs,
    vector_size=embedding_dim,
    window=5,
    min_count=1,
    sg=1,
)

In [18]:
# function to create embedding vector from tokens
def get_embedding(song_desc_tokens, model, embedding_dim):
    vectors = [model.wv[token] for token in song_desc_tokens if token in model.wv]

    # Average of word vectors OR zeros if no valid tokens found
    return sum(vectors) / len(vectors) if vectors else [0] * embedding_dim

In [19]:
# embed song descriptions as vectors
categorical_embeddings = [
    get_embedding(song_desc_tokens, word2Vec_model, embedding_dim)
    for song_desc_tokens in tokenised_song_descs
]

In [20]:
show_embeddings(categorical_embeddings)

Num Embeddings: 157685
Embedding Size: 15


[-1.4697373,
 1.6093767,
 1.3470358,
 -0.17075126,
 0.22816595,
 -0.2620371,
 -1.4705019,
 -0.023817305,
 -0.42897013,
 1.1646105,
 0.7519666,
 0.15207957,
 0.5491761,
 -0.3958861,
 -0.8389748]

### B. Embed Numeric Song Metadata

There are 15 numeric columns in our data which we will use to make up the other half of our final embedding vectors.
First, however, we will scale these values to make them more uniform.

The standard scaled score of a sample `x` is calculated as:

$$
    z = \frac{(x - u)}{s}
$$

where `u` is the mean of the training samples and `s` is the standard deviation of the training samples.

In [21]:
# extract numeric columns
numeric_cols = list(
    song_data.drop(columns=["song_name", "song_artists", "song_description"]).columns
)
numeric_cols

['song_acousticness',
 'song_danceability',
 'song_duration_ms',
 'song_energy',
 'song_explicit',
 'song_instrumentalness',
 'song_key',
 'song_liveness',
 'song_loudness',
 'song_mode',
 'song_popularity',
 'song_speechiness',
 'song_tempo',
 'song_valence',
 'song_year']

In [22]:
# scale these columns
scaled_numeric_cols = [
    (song_data[col] - song_data[col].mean()) / np.std(song_data[col])
    for col in numeric_cols
]

In [23]:
# transpose the array to get row embeddings
numeric_embeddings = list(map(list, zip(*scaled_numeric_cols)))

In [24]:
show_embeddings(numeric_embeddings)

Num Embeddings: 157685
Embedding Size: 15


[1.2703070294949268,
 -1.4612590488848611,
 4.752569009266431,
 -1.007676175100147,
 -0.30920114813582733,
 2.262496351076133,
 1.3649563314120978,
 2.611001210495346,
 -1.507817607982146,
 0.6453499264345925,
 -1.2499471942270766,
 -0.38364744367671405,
 -1.1655450558051323,
 -1.7786347004763179,
 -2.142666230649063]

### C. Merge Categorical & Numeric Embeddings

This leaves us with two sets of vectors: one representing the categorical column and one representing the numeric columns.
Both sets have 15 values each, so when we join these together, the resulting vector will have 30 values.

In [25]:
row_embeddings = [
    np.concatenate([cat_row, num_row])
    for cat_row, num_row in zip(categorical_embeddings, numeric_embeddings)
]

In [26]:
show_embeddings(row_embeddings)

Num Embeddings: 157685
Embedding Size: 30


[-1.4697372913360596,
 1.6093766689300537,
 1.3470357656478882,
 -0.17075125873088837,
 0.22816595435142517,
 -0.26203709840774536,
 -1.4705018997192383,
 -0.023817304521799088,
 -0.4289701282978058,
 1.164610505104065,
 0.7519665956497192,
 0.15207956731319427,
 0.5491760969161987,
 -0.395886093378067,
 -0.8389747738838196,
 1.2703070294949268,
 -1.4612590488848611,
 4.752569009266431,
 -1.007676175100147,
 -0.30920114813582733,
 2.262496351076133,
 1.3649563314120978,
 2.611001210495346,
 -1.507817607982146,
 0.6453499264345925,
 -1.2499471942270766,
 -0.38364744367671405,
 -1.1655450558051323,
 -1.7786347004763179,
 -2.142666230649063]

### Create DataFrame With Embeddings

We can take these defined embeddings and create a Pandas DataFrame containing them.
This will be the table we insert into our vector database.

To enable proper filtering of the data once inserted into the KDB.AI vector database, we will pair these embedding vectors with three song description columns: `song_name`, `song_artists`, and `song_year`.

In [27]:
embedded_song_df = song_data[["song_name", "song_artists", "song_year"]]

In [28]:
embedded_song_df["song_embeddings"] = row_embeddings

In [29]:
show_df(embedded_song_df)

(157685, 4)


,song_name,song_artists,song_year,song_embeddings
0,"Piano Concerto No. 3 in D Minor, Op. 30: III. Finale. Alla breve","Sergei Rachmaninoff, James Levine, Berliner Philharmoniker",1921,"[-1.4697372913360596, 1.6093766689300537, 1.3470357656478882, -0.17075125873088837, 0.22816595435142517, -0.26203709840774536, -1.4705018997192383, -0.023817304521799088, -0.4289701282978058, 1.164610505104065, 0.7519665956497192, 0.15207956731319427, 0.5491760969161987, -0.395886093378067, -0.8389747738838196, 1.2703070294949268, -1.4612590488848611, 4.752569009266431, -1.007676175100147, -0.30920114813582733, 2.262496351076133, 1.3649563314120978, 2.611001210495346, -1.507817607982146, 0.6453499264345925, -1.2499471942270766, -0.38364744367671405, -1.1655450558051323, -1.7786347004763179, -2.142666230649063]"
1,Clancy Lowered the Boom,Dennis Day,1921,"[-0.36899667978286743, 0.8569161295890808, 1.2207238674163818, 0.1085016131401062, 0.6271806359291077, 0.7028512954711914, -0.8539139032363892, 0.035000015050172806, -0.28415733575820923, 0.05096964165568352, 0.7450030446052551, 0.5746769309043884, 0.7175619006156921, -0.2206762284040451, 0.23307159543037415, 0.6055353575765623, 1.6000085277004752, -0.3958314054754826, -0.5219481676317392, -0.30920114813582733, -0.534994460237875, 0.5117210944218658, -0.2664040085915135, -0.16462578355006488, 0.6453499264345925, -1.2044048830282414, 1.8727935481311937, -1.8164526201211952, 1.654185186662974, -2.142666230649063]"
2,Gati Bali,KHP Kridhamardawa Karaton Ngayogyakarta Hadiningrat,1921,"[-0.5444679260253906, 0.5075699687004089, 0.04936423897743225, 0.20145872235298157, 0.9237421154975891, 0.27432358264923096, 0.20172667503356934, 0.08243125677108765, -0.3870060443878174, 0.07849937677383423, 0.7237778306007385, -0.012244322337210178, 0.18659956753253937, 0.5280500054359436, -1.1278307437896729, 1.214466209053784, -1.1834773613798955, 2.130627412712585, -1.1758127930699804, -0.30920114813582733, 2.3740136385430923, -0.6259258882317768, -0.6025761034947307, -0.5873232499193506, 0.6453499264345925, -1.2044048830282414, -0.39974784187407486, -0.21005905433507951, -1.854615663007068, -2.142666230649063]"
3,Danny Boy,Frank Parker,1921,"[-0.39720410108566284, 1.1563446521759033, 1.569617509841919, 0.1184227466583252, 0.3758953809738159, 0.6490341424942017, -1.0218818187713623, 0.11120273917913437, -0.36480364203453064, 0.28904595971107483, 0.9119070172309875, 0.5089620351791382, 1.1516392230987549, -0.19770382344722748, 0.3800709545612335, 1.230420729179825, -1.4839351050077156, -0.1628410916211915, -0.641511984854732, -0.30920114813582733, -0.5349062022703654, -0.057102396904955506, 0.9928168892663002, 0.38370530088497806, 0.6453499264345925, -1.295489505425912, -0.3908031762088744, -0.5426988976237861, -1.377455218313957, -2.142666230649063]"
4,When Irish Eyes Are Smiling,Phil Regan,1921,"[-0.26264292001724243, 0.8120734095573425, 0.90312260389328, 0.13470879197120667, 0.8935577869415283, 0.6051231622695923, -1.26597261428833, 0.3834632635116577, -0.1906394511461258, -0.06401898711919785, 1.3381379842758179, 0.31093665957450867, 0.8109012842178345, -0.344455748796463, 0.09799161553382874, 1.2038298623030903, -0.6732660986156729, -0.5052618172494464, -1.0749308222880805, -0.30920114813582733, -0.5349891074080766, -0.6259258882317768, 0.12674640748174057, 0.24684186220999144, 0.6453499264345925, -1.341031816624747, -0.3752990890558602, -0.49210382468564035, -1.0431389831786564, -2.142666230649063]"


## 3. Store Embeddings In KDB.AI

Now that our data has been read in and our embeddings have been defined, we can set about creating a KDB.AI vector database and ingesting the data into it.

Once we have the database set up, we can then query our data and start deriving some insights

### Connect to KDB.AI Session

To use KDB.AI, you will need two session details - a URL endpoint and an API key. To get these you can sign up for free [here](https://trykdb.kx.com/kdbai/signup).

You can connect to a KDB.AI session using `kdbai.Session` and passing the session URL endpoint and API key details from your KDB.AI Cloud portal.

If the environment variables `KDBAI_ENDPOINTS` and `KDBAI_API_KEY` exist on your system containing your KDB.AI Cloud portal details, these variables will automatically be used to connect.
If these do not exist, it will prompt you to enter your KDB.AI Cloud portal session URL endpoint and API key details.

In [30]:
KDBAI_ENDPOINT = (
    os.environ["KDBAI_ENDPOINT"]
    if "KDBAI_ENDPOINT" in os.environ
    else input("KDB.AI endpoint: ")
)
KDBAI_API_KEY = (
    os.environ["KDBAI_API_KEY"]
    if "KDBAI_API_KEY" in os.environ
    else getpass("KDB.AI API key: ")
)

In [31]:
session = kdbai.Session(api_key=KDBAI_API_KEY, endpoint=KDBAI_ENDPOINT)

We can check if there are any tables defined in this KDB.AI session - it should not contain a table called `songs`.

In [32]:
session.list()

[]

### Define Vector DB Table Schema

The next step is to define the schema for the table in KDB.AI which will store our embeddings.

As mentioned above, our table will have four columns:
- Song Name
- Song Artists
- Song Year
- Song Embeddings

When defining the schema, we must supply the types of these columns. We can use the `.dtypes()` function on the defined Pandas DataFrame to help with this.

In [33]:
embedded_song_df.dtypes

song_name          object
song_artists       object
song_year           int64
song_embeddings    object
dtype: object

In [34]:
schema = {
    "columns": [
        {
            "name": "song_name",
            "pytype": "str",
        },
        {
            "name": "song_artists",
            "pytype": "str",
        },
        {
            "name": "song_year",
            "pytype": "int64",
        },
        {
            "name": "song_embeddings",
            "vectorIndex": {
                "dims": len(numeric_cols) + embedding_dim,
                "metric": "L2",
                "type": "flat",
            },
        },
    ]
}

### Create Vector DB Table

Use the KDB.AI `create_table` function to create a table that matches the defined schema in the vector database.

In [35]:
# First ensure the table does not already exist
try:
    session.table("songs").drop()
    time.sleep(5)
except kdbai.KDBAIException:
    pass

In [36]:
table = session.create_table("songs", schema)

### Add Embedded Data to KDB.AI Table

When adding larger amounts of data, you should insert data into an index in chunks.

It is a good idea to first get an idea of how large your dataset to insert is.

In [37]:
embedded_song_df.memory_usage(deep=True).sum() / (1024**2)

79.63715362548828

This dataset is 80MB which exceeds the insert limit of <10MB at a time. As such, we'll insert this data in chunks, inserting 10,000 rows at a time.

In [38]:
chunk_size = 10_000

In [39]:
for i in range((len(embedded_song_df) // chunk_size) + 1):
    index = i * chunk_size
    table.insert(
        embedded_song_df.iloc[index : index + chunk_size].reset_index(drop=True)
    )

### Verify Data Has Been Inserted

Running `table.query()` should show us that data has been added.

In [40]:
show_df(table.query())

(157685, 4)


,song_name,song_artists,song_year,song_embeddings
0,"Piano Concerto No. 3 in D Minor, Op. 30: III. Finale. Alla breve","Sergei Rachmaninoff, James Levine, Berliner Philharmoniker",1921,"[-1.4697372913360596, 1.6093766689300537, 1.3470357656478882, -0.17075125873088837, 0.22816595435142517, -0.26203709840774536, -1.4705018997192383, -0.023817304521799088, -0.4289701282978058, 1.164610505104065, 0.7519665956497192, 0.15207956731319427, 0.5491760969161987, -0.395886093378067, -0.8389747738838196, 1.2703070294949268, -1.4612590488848611, 4.752569009266431, -1.007676175100147, -0.30920114813582733, 2.262496351076133, 1.3649563314120978, 2.611001210495346, -1.507817607982146, 0.6453499264345925, -1.2499471942270766, -0.38364744367671405, -1.1655450558051323, -1.7786347004763179, -2.142666230649063]"
1,Clancy Lowered the Boom,Dennis Day,1921,"[-0.36899667978286743, 0.8569161295890808, 1.2207238674163818, 0.1085016131401062, 0.6271806359291077, 0.7028512954711914, -0.8539139032363892, 0.035000015050172806, -0.28415733575820923, 0.05096964165568352, 0.7450030446052551, 0.5746769309043884, 0.7175619006156921, -0.2206762284040451, 0.23307159543037415, 0.6055353575765623, 1.6000085277004752, -0.3958314054754826, -0.5219481676317392, -0.30920114813582733, -0.534994460237875, 0.5117210944218658, -0.2664040085915135, -0.16462578355006488, 0.6453499264345925, -1.2044048830282414, 1.8727935481311937, -1.8164526201211952, 1.654185186662974, -2.142666230649063]"
2,Gati Bali,KHP Kridhamardawa Karaton Ngayogyakarta Hadiningrat,1921,"[-0.5444679260253906, 0.5075699687004089, 0.04936423897743225, 0.20145872235298157, 0.9237421154975891, 0.27432358264923096, 0.20172667503356934, 0.08243125677108765, -0.3870060443878174, 0.07849937677383423, 0.7237778306007385, -0.012244322337210178, 0.18659956753253937, 0.5280500054359436, -1.1278307437896729, 1.214466209053784, -1.1834773613798955, 2.130627412712585, -1.1758127930699804, -0.30920114813582733, 2.3740136385430923, -0.6259258882317768, -0.6025761034947307, -0.5873232499193506, 0.6453499264345925, -1.2044048830282414, -0.39974784187407486, -0.21005905433507951, -1.854615663007068, -2.142666230649063]"
3,Danny Boy,Frank Parker,1921,"[-0.39720410108566284, 1.1563446521759033, 1.569617509841919, 0.1184227466583252, 0.3758953809738159, 0.6490341424942017, -1.0218818187713623, 0.11120273917913437, -0.36480364203453064, 0.28904595971107483, 0.9119070172309875, 0.5089620351791382, 1.1516392230987549, -0.19770382344722748, 0.3800709545612335, 1.230420729179825, -1.4839351050077156, -0.1628410916211915, -0.641511984854732, -0.30920114813582733, -0.5349062022703654, -0.057102396904955506, 0.9928168892663002, 0.38370530088497806, 0.6453499264345925, -1.295489505425912, -0.3908031762088744, -0.5426988976237861, -1.377455218313957, -2.142666230649063]"
4,When Irish Eyes Are Smiling,Phil Regan,1921,"[-0.26264292001724243, 0.8120734095573425, 0.90312260389328, 0.13470879197120667, 0.8935577869415283, 0.6051231622695923, -1.26597261428833, 0.3834632635116577, -0.1906394511461258, -0.06401898711919785, 1.3381379842758179, 0.31093665957450867, 0.8109012842178345, -0.344455748796463, 0.09799161553382874, 1.2038298623030903, -0.6732660986156729, -0.5052618172494464, -1.0749308222880805, -0.30920114813582733, -0.5349891074080766, -0.6259258882317768, 0.12674640748174057, 0.24684186220999144, 0.6453499264345925, -1.341031816624747, -0.3752990890558602, -0.49210382468564035, -1.0431389831786564, -2.142666230649063]"


## 4. Search For Similar Songs To A Target Song

Now that the data has been inserted into the database, we can perform some queries on the data.

### Find Songs By A Certain Artist

We can query the database to find songs by particular artists using KDB.AI's `.query()` function.

Here, we want to return all songs in the dataset by the DJ `Calvin Harris`, sorted by the year they were produced. This returns 32 songs to us.

In [41]:
table.query(filter=[("like", "song_artists", "*Calvin Harris*")], sort_by="song_year")

,song_name,song_artists,song_year,song_embeddings
0,Flashback,Calvin Harris,2009,"[-0.4177802801132202, 1.1748639345169067, 0.6881563663482666, 0.013721980154514313, 0.9479737281799316, 0.3897114396095276, -0.6866428852081299, -0.019070934504270554, 0.014673478901386261, 0.21349138021469116, 0.4838828444480896, 0.747475266456604, 1.0940808057785034, -0.06278841197490692, 0.4147791266441345, -1.3351724705550345, -1.5406252453148515, -0.011030115749796733, 1.7460279903169027, -0.30920114813582733, -0.531394044956799, 1.080544585748687, -0.7039975151773963, 1.0250333372402045, -1.5495469342051147, 0.7539144985216722, -0.3073196300003366, 0.36420802995835594, -1.2520866301382192, 1.24780896840589]"
1,You Used To Hold Me,Calvin Harris,2009,"[-0.1804826855659485, 1.3096168041229248, 1.0537090301513672, -0.13217858970165253, 0.7977455258369446, 0.44970786571502686, -1.2015180587768555, 0.3313247561454773, 0.05825001373887062, 0.05978407338261604, 1.5313469171524048, 0.1717090904712677, 1.2571277618408203, 0.007039185613393784, 0.279476523399353, -1.2794911953151524, 0.46053670752704456, 0.0042537705189223335, 1.8244917453694918, -0.30920114813582733, -0.5349904137534441, 1.6493680770755084, 0.6224578016610608, 0.9587072092669416, -1.5495469342051147, 0.5717452537263314, -0.3556208245924191, 0.39555356357813237, -1.2824790151505194, 1.24780896840589]"
2,I'm Not Alone - Radio Edit,Calvin Harris,2009,"[-0.3499296307563782, 1.244738221168518, 0.9763027429580688, -0.14675182104110718, 0.7054509520530701, 0.5618191957473755, -1.0987207889556885, 0.4011222720146179, 0.022020503878593445, 0.24286241829395294, 1.2122381925582886, 0.3463975787162781, 1.0734851360321045, -0.2579449713230133, 0.26039785146713257, -1.3273547556932748, 0.30747332869777755, -0.14913856968446612, 0.7932538218211799, -0.30920114813582733, -0.14309084999684657, 0.5117210944218658, 0.491407662969976, 0.8665875870818543, 0.6453499264345925, 0.3895760089309906, -0.4110777517166621, 0.46191895166730623, -0.3631093685284425, 1.24780896840589]"
3,We Found Love,"Rihanna, Calvin Harris",2011,"[-0.22046250104904175, 1.4409897327423096, 1.0603516101837158, -0.15190696716308594, 0.7062567472457886, 0.4542839527130127, -1.1391438245773315, 0.2879817485809326, 0.00519922748208046, 0.17919489741325378, 1.0258654356002808, 0.5709000825881958, 1.2584115266799927, -0.15058273077011108, 0.18762657046318054, -1.2744389306085728, 1.1181423350898205, -0.12151213480453628, 1.0660087798611322, -0.30920114813582733, -0.5305974929034635, -1.1947493795585982, -0.562691278675705, 1.2313812909347996, 0.6453499264345925, 1.9835569008902227, -0.3735101559228201, 0.36375280436636764, 0.2751307167298588, 1.324865222929866]"
4,Dance Wiv Me - Radio Edit,"Dizzee Rascal, Calvin Harris, Chrome",2011,"[-0.4587083160877228, 1.0718482732772827, 0.7888512015342712, -0.19494929909706116, 0.5652546286582947, 0.5162851810455322, -0.7239663004875183, 0.22776807844638824, -0.27129143476486206, 0.2589092552661896, 0.6417447924613953, 0.29954248666763306, 0.8052375912666321, -0.016278591006994247, 0.2674573063850403, -1.2143435714671529, 1.9344803555125771, -0.2095466871666253, 0.9912813940967616, -0.30920114813582733, -0.534994460237875, 1.6493680770755084, -0.3005910012935356, 1.2671763441267194, 0.6453499264345925, 1.5281337889018707, -0.3329610049072446, -0.1561798539118747, 1.0045479570250604, 1.324865222929866]"
5,Feel So Close - Radio Edit,Calvin Harris,2012,"[-0.3422718048095703, 1.2694331407546997, 0.8576171398162842, -0.13317301869392395, 0.7552549839019775, 0.6506489515304565, -1.012566328048706, 0.3336513340473175, -0.20709870755672455, 0.23876579105854034, 1.074584722518921, 0.40791648626327515, 1.2946882247924805, -0.191492959856987, 0.43210864067077637, -1.3383314655399907, 0.9650789562605536, -0.19120286091549843, 1.6563551273996584, -0.30920114813582733, -0.5125954164980829, 0.5117210944218658, -0.015699395443351603, 1.5196718418873678, 0.6453499264345925, 2.2112684568843988, -0.41704086216012914, 

### Find A Specific Song

We can filter this query further by looking for the song `We Found Love` by `Calvin Harris` in our dataset.

This will only return one song as he only produced one song with this name.

In [42]:
table.query(
    filter=[
        ("like", "song_artists", "*Calvin Harris*"),
        ("like", "song_name", "*We Found Love*"),
    ]
)

,song_name,song_artists,song_year,song_embeddings
0,We Found Love,"Rihanna, Calvin Harris",2011,"[-0.22046250104904175, 1.4409897327423096, 1.0603516101837158, -0.15190696716308594, 0.7062567472457886, 0.4542839527130127, -1.1391438245773315, 0.2879817485809326, 0.00519922748208046, 0.17919489741325378, 1.0258654356002808, 0.5709000825881958, 1.2584115266799927, -0.15058273077011108, 0.18762657046318054, -1.2744389306085728, 1.1181423350898205, -0.12151213480453628, 1.0660087798611322, -0.30920114813582733, -0.5305974929034635, -1.1947493795585982, -0.562691278675705, 1.2313812909347996, 0.6453499264345925, 1.9835569008902227, -0.3735101559228201, 0.36375280436636764, 0.2751307167298588, 1.324865222929866]"


### Find Similar Songs To This Song

We can then copy and paste the vector associated with this song below and save it as the variable `my_vec`.

We will then use KDB.AI's `.search()` function to find similar songs in the dataset to this song using this vector.
We will pull out the 5 songs most similar to this song from the dataset.

<div class="alert alert-block alert-warning">
    <b>Note: </b>
    The most similar song will be "We Found Love" by "Calvin Harris" as this is the vector we are using for the search.
</div>

In [43]:
my_vec = [-0.22046250104904175, 1.4409897327423096, 1.0603516101837158, -0.15190696716308594, 0.7062567472457886, 0.4542839527130127, -1.1391438245773315, 0.2879817485809326, 0.00519922748208046, 0.17919489741325378, 1.0258654356002808, 0.5709000825881958, 1.2584115266799927, -0.15058273077011108, 0.18762657046318054, -1.2744389306085728, 1.1181423350898205, -0.12151213480453628, 1.0660087798611322, -0.30920114813582733, -0.5305974929034635, -1.1947493795585982, -0.562691278675705, 1.2313812909347996, 0.6453499264345925, 1.9835569008902227, -0.3735101559228201, 0.36375280436636764, 0.2751307167298588, 1.324865222929866]

In [44]:
table.search(vectors=[my_vec], n=5)[0]

,song_name,song_artists,song_year,song_embeddings,__nn_distance
0,We Found Love,"Rihanna, Calvin Harris",2011,"[-0.22046250104904175, 1.4409897327423096, 1.0603516101837158, -0.15190696716308594, 0.7062567472457886, 0.4542839527130127, -1.1391438245773315, 0.2879817485809326, 0.00519922748208046, 0.17919489741325378, 1.0258654356002808, 0.5709000825881958, 1.2584115266799927, -0.15058273077011108, 0.18762657046318054, -1.2744389306085728, 1.1181423350898205, -0.12151213480453628, 1.0660087798611322, -0.30920114813582733, -0.5305974929034635, -1.1947493795585982, -0.562691278675705, 1.2313812909347996, 0.6453499264345925, 1.9835569008902227, -0.3735101559228201, 0.36375280436636764, 0.2751307167298588, 1.324865222929866]",0.000000
1,Bad At Love,Halsey,2017,"[-0.20234839618206024, 1.2302510738372803, 1.3452625274658203, 0.19579169154167175, 0.3654645085334778, 0.6483091115951538, -1.0885369777679443, 0.4906502664089203, -0.1202155128121376, 0.16909970343112946, 1.3230335712432861, 0.5799967050552368, 0.8773004412651062, -0.06735996901988983, 0.3701416254043579, -1.1803072618649324, 0.7836705072777193, -0.3899329165171462, 1.0099632405378542, -0.30920114813582733, -0.534994460237875, -1.4791611252220087, -0.6692407392636738, 1.397372076814976, 0.6453499264345925, 1.9380145896913874, -0.4253892167809829, 0.051533080489714604, 0.3207192942483089, 1.5560339865017947]",1.207436
2,Sweet but Psycho,Ava Max,2020,"[-0.22328363358974457, 1.0351508855819702, 0.9286673069000244, -0.19666792452335358, 0.6260932087898254, 0.5847731232643127, -1.0783933401107788, 0.35929977893829346, -0.19377990067005157, 0.5673081278800964, 0.9074316620826721, 0.620949923992157, 0.761772096157074, -0.33370736241340637, 0.021074196323752403, -1.1590345683635448, 1.0387761386598302, -0.34125061555672026, 0.8418266225680207, -0.30920114813582733, -0.534994460237875, -1.1947493795585982, -0.23221701588949137, 1.1903222593323037, 0.6453499264345925, 2.3934377016797397, -0.31984216193161724, 0.5268536307530883, 0.351111679260609, 1.671618368287759]",1.283543
3,No One Compares To You,Jack & Jack,2019,"[-0.3724234402179718, 1.1231266260147095, 1.107741355895996, -0.12134488672018051, 0.4830331802368164, 0.33535465598106384, -1.0928318500518799, 0.0477365180850029, -0.20143654942512512, 0.27681055665016174, 1.4722427129745483, 0.39361679553985596, 0.8545140027999878, -0.1336672306060791, 0.26146382093429565, -1.0032120884658802, 0.9253958580455585, -0.3769578221731379, 0.9987541326731987, -0.30920114813582733, -0.534994460237875, -1.1947493795585982, -0.6418911451020561, 0.9578298795318455, 0.6453499264345925, 1.8924722784925523, -0.3430982926611385, -0.15803327239354203, 0.3929012086525216, 1.6330902410257708]",1.354893
4,Stay Gold,BTS,2020,"[-0.20532099902629852, 1.0667123794555664, 0.8290035128593445, -0.034996382892131805, 0.5100072026252747, 0.7065194249153137, -1.0525459051132202, 0.2753138542175293, -0.44233986735343933, 0.25172123312950134, 1.1644545793533325, 0.42643532156944275, 0.771487832069397, -0.2981712222099304, 0.29280340671539307, -1.103725565259937, 1.055783180751971, 0.10019514454097984, 0.5728080338162873, -0.30920114813582733, -0.534994460237875, -1.1947493795585982, -0.7296377597039129, 1.068197960206931, 0.6453499264345925, 2.2112684568843988, -0.27869669987169504, 0.3972769176049457, 0.20674785045218355, 1.671618368287759]",1.399699


### Automate This Song Similarity Search Process

We can define a function to automate this process and find songs which are the most similar to any input song.
This will allow us to use the KDB.AI vector database in a more production-like setting to perform similarity search and music recommendation.

In [45]:
def find_similar_songs(
    vectorDB_song_tab,
    song_name: str,
    song_artists: list[str] = None,
    song_year: int = None,
    n_similar: int = 5,
    exact: bool = False,
) -> None:
    # create filter list
    filter_list = [("like", "song_name", f"{song_name}" if exact else f"*{song_name}*")]
    if song_artists:
        if type(song_artists) == str:
            song_artists = list(song_artists)
        for artist in song_artists:
            filter_list.append(("like", "song_artists", f"*{artist}*"))
    if song_year:
        filter_list.append(("like", "song_year", f"{song_year}"))

    # find songs liks this in vector DB
    resulting_song = vectorDB_song_tab.query(filter=filter_list, sort_by="song_year")

    # quality check
    if resulting_song.empty:
        print(
            "Song Not Found! Please double check the values entered or try another song"
        )
        return

    # find vectors associated with these songs
    resulting_vectors = [v.tolist() for v in resulting_song["song_embeddings"]]

    # search for similar songs to selected songs
    similar_songs = vectorDB_song_tab.search(vectors=resulting_vectors, n=n_similar + 1)

    # process similar song table
    for i, similar_df in enumerate(similar_songs):
        name = resulting_song.loc[i, "song_name"]
        artists = resulting_song.loc[i, "song_artists"]
        year = resulting_song.loc[i, "song_year"]
        print(f"Songs Similar To '{name}' By '{artists}' ({year})")
        for j, song in similar_df[1:].iterrows():
            print(
                f"   {j}. {song['song_name']} - {song['song_artists']} ({song['song_year']})"
            )
        print()

##### Songs by multiple artists

Here, we will query this function to search the KDB.AI vector database to look for the song `Let's Go` by two artists - `Calvin Harris` and `Ne-Yo`. 

In [46]:
find_similar_songs(table, song_name="Let's Go", song_artists=["Calvin Harris", "Ne-Yo"])

Songs Similar To 'Let's Go (feat. Ne-Yo)' By 'Calvin Harris, Ne-Yo' (2012)
   1. I Cry - Flo Rida (2012)
   2. Mmm Yeah (feat. Pitbull) - Austin Mahone, Pitbull (2014)
   3. Too Much (feat. Usher) - Marshmello, Imanbek, Usher (2020)
   4. All Around The World - Justin Bieber, Ludacris (2012)
   5. No Money - Galantis (2016)



If you search for these songs on [YouTube](https://www.youtube.com/), you will see that the results returned have quite a similar vibe to the song we searched for showing the similarity search power of KDB.AI.

##### Specify different number of similar songs

We can adjust the number of results returned to us by specifying the `n_similar` parameter.
Here, we will search for the `8` most similar songs to the song `Californiacation` by `Red Hot Chili Peppers`.

In [47]:
find_similar_songs(
    table,
    song_name="Californication",
    song_artists="Red Hot Chili Peppers",
    n_similar=8,
)

Songs Similar To 'Californication' By 'Red Hot Chili Peppers' (1999)
   1. Police Station - Red Hot Chili Peppers (2011)
   2. Charlie - Red Hot Chili Peppers (2006)
   3. Dark Necessities - Red Hot Chili Peppers (2016)
   4. Especially in Michigan - Red Hot Chili Peppers (2006)
   5. Don't Forget Me - Red Hot Chili Peppers (2002)
   6. Cabron - Red Hot Chili Peppers (2002)
   7. Look Around - Red Hot Chili Peppers (2011)
   8. Midnight City - M83 (2011)



When we return these similar songs, you will notice that a lot of the similar songs are by the same artist as the original song we searched for - `Red Hot Chili Peppers`.
This makes sense as their music usually has pretty unique features. 

#### All songs with a given name by any artist

The final thing we will perform a similarity search on all songs stored in the KDB.AI vector database with a given song name - `Love Me`.

In [48]:
find_similar_songs(table, song_name="Love Me", exact=True)

Songs Similar To 'Love Me' By 'Elvis Presley' (1956)
   1. Don't - Elvis Presley (1959)
   2. Without Him - Elvis Presley (1967)
   3. Everything I Have Is Yours - 10'' Version - Billie Holiday (1956)
   4. His Hand in Mine - Elvis Presley (1960)
   5. Fine And Mellow - Billie Holiday (1957)

Songs Similar To 'Love Me' By 'Buddy Holly' (1958)
   1. Midnight Shift - Buddy Holly (1958)
   2. Johnny be good - Radio Version - Jonny Bombastic (1955)
   3. A Love That's Worth Having - Willie Hutch (1969)
   4. Lonely Weekends - Wanda Jackson (1961)
   5. Rock & Roll Guitar - Johnny Knight (1959)

Songs Similar To 'Love Me' By 'Sarah Vaughan' (1958)
   1. Summer Is Gone - Carmen McRae (1956)
   2. Make the World Go Away - Ray Price (1956)
   3. Your Love Has Faded - Johnny Hodges (1961)
   4. I'll Come Back For More - Dinah Washington (1961)
   5. I'm Confessin' (That I Love You) - Judy Garland (1958)

Songs Similar To 'Love Me' By 'Bo Diddley' (1960)
   1. Keepin' Out Of Mischief Now - Barbr

There are 15 songs in our vector database with `Love Me` as their title and we have returned the most similar songs to each of these.
With that, we have built a recommendation system which is able to recommend music based both on user numerical and categorical song data.

## 5. Delete the KDB.AI Table

Once finished with the table, it is best practice to drop it.

In [49]:
table.drop()

True